# PYNQ MD5 driver v C

In [1]:
import numpy as np
from pynq import allocate, Overlay, Clocks
from cffi import FFI
import hashlib

ffi = FFI()

with open("preprocess.c") as f:
    c_code = f.read()

ffi.cdef("""
    void preprocess(unsigned char * buffer_out, unsigned char **inputs, int *sizes);
""")
C = ffi.verify(c_code)

# Define custom types
uint512 = np.dtype((np.uint8, 64))
uint128 = np.dtype((np.uint8, 16))

def print_uint512(arr):
    for i in range(len(arr)):
        print(f'uint512[{i}] = ', end='')
        for part in arr[i]:
            print(f'{part:02x}', end='')
        print()
        
def print_uint128(arr):
    for i in range(len(arr)):
        print(f'uint128[{i}] = ', end='')
        for part in arr[i]:
            print(f'{part:02x}', end='')
        print()
        
class MD5Overlay(Overlay):
    def __init__(self, bitfile, **kwargs):
        super().__init__(bitfile, **kwargs)
    
    
    def md5(self, data_list):
                
        # Pripravi pointerje na vhode in na velikosti teh vhodov (omejitev: vsi morajo biti enako veliki)
        data_list_ptrs = ffi.new("unsigned char*[]", [ffi.from_buffer(l) for l in data_list])
        sizes = [len(data_list[0])]*64
        sizes_ptr = ffi.new("int[]", sizes)
        
        # Pripravi buffer v katerega bo C zapisal procesirane podatke
        size = (((sizes[0]+9) + 63) // 64) * 64 * 64
        buffer_out = ffi.new("unsigned char[]", size)
        
        C.preprocess(buffer_out, data_list_ptrs, sizes_ptr)
        interleaved = np.frombuffer(ffi.buffer(buffer_out, size), dtype=uint512)
        
        input_buffer = allocate(shape=(np.shape(interleaved)[0],), dtype=uint512)
        np.copyto(input_buffer, interleaved)
                
        output_buffer = allocate(shape=(64,), dtype=uint128)
        dma = self.axi_dma_0
        dma.sendchannel.transfer(input_buffer)
        dma.recvchannel.transfer(output_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        
        return output_buffer

In [2]:
overlay = MD5Overlay('/home/xilinx/pynq/overlays/pynq-MD5_2/pynq-MD5_2.bit')
Clocks.fclk0_mhz = 250

data_list = [bytearray([i+97] * 102400) for i in range(64)]

/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/ps.py:434: UserWarning: Setting frequency to the closest possible value 98.8879MHz.
  warnings.warn(
/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/ps.py:434: UserWarning: Setting frequency to the closest possible value 247.21975MHz.
  warnings.warn(


In [3]:
%%timeit

overlay.md5(data_list)
#print_uint128(hash)

29.9 ms ± 69.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit

for data in data_list:
    hashlib.md5(data)

40.2 ms ± 52.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
